## Script for conducting ISC on emotion ECG data

###  Section 1: Basic setting & Data Import

In [ ]:
%matplotlib inline
import os
import neurokit2 as nk
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import rankdata
from sklearn.metrics import pairwise_distances
from nltools.stats import isc
from nltools.data import Adjacency
from Function import import_ecg, rain_ecg, rain_hrv, mean_df2_phy, mean_df3_phy
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(theme='grade3') 

### Subject list
sub_ag = [1, 2] + list(range(4,12)) + [13,14,16,17] + list(range(19,24)) + [25,26,28,29,30,31] + list(range(33,43)) + [46] # 36
sub_ax = list(range(1,5)) + list(range(7,17)) + [19,20,21,22,23,25,26,28,29,30,31] + list(range(33,42)) + [43,44,45,46] # 38
sub_fe = list(range(1,13)) + [14,17,19,20,21,22,23] + [25,26,28,29,30,31] + list(range(33,41)) + [42,44,45,46] # 37
sub_hl = [1,2] + list(range(4,18)) + [19,20,21,22,23,25,26,28,29,30,31] + list(range(33,42)) + [43,44,46] # 39
sub_ha = list(range(1,18)) + [19,20,21,22,23,25,26,28,29,31] + list(range(33,47)) # 41

In [ ]:
# Define the directory
ag_dir = 'F:/1_Emotion_Data/Data/Physiology/Angry/'
ax_dir = 'F:/1_Emotion_Data/Data/Physiology/Anxiety/'
fe_dir = 'F:/1_Emotion_Data/Data/Physiology/Fear/'
hl_dir = 'F:/1_Emotion_Data/Data/Physiology/Helpless/'
ha_dir = 'F:/1_Emotion_Data/Data/Physiology/Happy/'
results_dir = 'F:/1_Emotion_Data/Results/0_ISC/Physiology/'

In [ ]:
## Import the flattened 1-D array
ag1_meta = import_ecg(sub_ag, 'angry', '1', ag_dir)
ag2_meta = import_ecg(sub_ag, 'angry', '2', ag_dir)
ag3_meta = import_ecg(sub_ag, 'angry', '3', ag_dir)
ax1_meta = import_ecg(sub_ax, 'anxiety', '1', ax_dir)
ax2_meta = import_ecg(sub_ax, 'anxiety', '2', ax_dir)
fe1_meta = import_ecg(sub_fe, 'fear', '1', fe_dir)
fe2_meta = import_ecg(sub_fe, 'fear', '2', fe_dir)
fe3_meta = import_ecg(sub_fe, 'fear', '3', fe_dir)
hl2_meta = import_ecg(sub_hl, 'helpless', '2', hl_dir)
hl3_meta = import_ecg(sub_hl, 'helpless', '3', hl_dir)
ha1_meta = import_ecg(sub_ha, 'happy', '1', ha_dir)
ha3_meta = import_ecg(sub_ha, 'happy', '3', ha_dir)

### Section 2: ISC for *Angry* ECG

* ag1: (1) 220-240, the headmaster mangae to rap the young girl
* ag2: (1) 125 - 145, Japanese army kill armless ordinary Chinese people
* ag3: (1) 65-85, the man beat the boy rudely  

 Preprocess the raw ECG and aggregate results
* Features: Cleaned signal, Heart Rate, ECG derived Respiration(EDR), HVR_MeanNN, HRV_SDNN

In [ ]:
# Construct a meta dict for angry results
ag_ecg, ag_hrv = {},{}
# Loop for all subjects
for sub in sub_ag:
    ecg_ag1, ecg_ag2, ecg_ag3 = ag1_meta[sub][220*2000:240*2000], ag2_meta[sub][125*2000:145*2000], ag3_meta[sub][65*2000:85*2000]
    ag1_results, ag2_results, ag3_results = rain_ecg(ecg_ag1, 2000, 250), rain_ecg(ecg_ag2, 2000, 250), rain_ecg(ecg_ag3, 2000, 250)
    ag1 = ag1_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    ag2 = ag2_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    ag3 = ag3_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    ag_ecg[sub] = mean_df3_phy(ag1, ag2, ag3)
    ag_hrv[sub] = mean_df3_phy(rain_hrv(ecg_ag1, 2000, 250), rain_hrv(ecg_ag2, 2000, 250), rain_hrv(ecg_ag3, 2000, 250))

## Save the angry result to .npy file
np.save(os.path.join(results_dir, 'ag_ecg.npy'), ag_ecg) 
np.save(os.path.join(results_dir, 'ag_hrv.npy'), ag_hrv) 

Extract features to conduct ISC

In [ ]:
ag_ecg = np.load(os.path.join(results_dir, 'ag_ecg.npy'), allow_pickle=True).item()
ag_hrv = np.load(os.path.join(results_dir, 'ag_hrv.npy'), allow_pickle=True).item()
ag_hr, ag_edr, ag_hrvm, ag_hrvsd = np.zeros((5000, 36)), np.zeros((5000, 36)), np.zeros((36)), np.zeros((36))
for sub in sub_ag:
    ## For the index 1 for 'ECG_Rate', 2 for 'EDR'
    hr = ag_ecg[sub].iloc[:,1].to_numpy()
    edr = ag_ecg[sub].iloc[:,2].to_numpy()
    mean = ag_hrv[sub].iloc[:,0]
    sd = ag_hrv[sub].iloc[:,1]
    hr = np.ndarray.flatten(hr)
    edr = np.ndarray.flatten(edr)
    idx = sub_ag.index(sub)
    ag_hr[:,idx] = hr
    ag_edr[:,idx] = edr
    ag_hrvm[idx] = mean
    ag_hrvsd[idx] = sd

Calculate ISC on Angry ECG features

In [ ]:
# Calculate ISC on angry heart rate
ag_hr_similarity, ag_hr_stats = isc(ag_hr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
ag_hr_similarity = ag_hr_similarity.squareform()
ag_edr_similarity, ag_edr_stats = isc(ag_edr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
ag_edr_similarity = ag_edr_similarity.squareform()

# Plot
mask =np.zeros_like(ag_hr_similarity)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(ag_hr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Angry HR ISC", fontsize=25, fontweight='bold')
sns.heatmap(ag_edr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Angry EDR ISC", fontsize=25, fontweight='bold')

# Save the result
np.save(os.path.join(results_dir, 'ag_hr_isc.npy'), ag_hr_similarity) 
np.save(os.path.join(results_dir, 'ag_edr_isc.npy'), ag_edr_similarity) 


Calculate ISC on Angry HRV features

In [ ]:
# Set the paprameters and import data
n_subs = 36
hrvm_rank, hrvsd_rank = rankdata(ag_hrvm), rankdata(ag_hrvsd) # explicity convert the raw scores to ranks
hrvm_simi, hrvsd_simi = np.zeros((n_subs, n_subs)), np.zeros((n_subs, n_subs))

# Calculate the ann isc
for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvm_rank[i], hrvm_rank[j]])/n_subs
            hrvm_simi[i,j] = sim_ij
            hrvm_simi[j,i] = sim_ij
        elif i==j:
            hrvm_simi[i,j] = 1

for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvsd_rank[i], hrvsd_rank[j]])/n_subs
            hrvsd_simi[i,j] = sim_ij
            hrvsd_simi[j,i] = sim_ij
        elif i==j:
            hrvsd_simi[i,j] = 1

# Plot the isc matrix
mask =np.zeros_like(hrvm_simi)
mask[np.triu_indices_from(mask)] = True
# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
# sns.heatmap(hrvm_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax1.set_title("Angry HRV mean ISC", fontsize=25, fontweight='bold')
# sns.heatmap(hrvsd_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax2.set_title("Angry HRV SD ISC", fontsize=25, fontweight='bold')

plt.figure(figsize=(20,10))
sns.heatmap(hrvm_simi, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
#plt.title('Angry HRV Intersubject Similarity', fontsize=25, fontweight='bold')
# Save the result
# np.save(os.path.join(results_dir, 'ag_hrvm_isc.npy'), hrvm_simi) 
# np.save(os.path.join(results_dir, 'ag_hrvsd_isc.npy'), hrvsd_simi) 

## Section 3: ISC for *Anxiety* EEG

* ax1: 135 - 155, The bomb is about to explode
* ax2: 135 - 155, The girl ask for the man to open the door


 Preprocess the raw ECG and aggregate results
* Features: Cleaned signal, Heart Rate, ECG derived Respiration(EDR), HVR_MeanNN, HRV_SDNN

In [ ]:
# Construct a meta dict for anxiety results
ax_ecg, ax_hrv = {},{}
# Loop for all subjects
for sub in sub_ax:
    ecg_ax1, ecg_ax2 = ax1_meta[sub][135*2000:155*2000], ax2_meta[sub][135*2000:155*2000]
    ax1_results, ax2_results = rain_ecg(ecg_ax1, 2000, 250), rain_ecg(ecg_ax2, 2000, 250)
    ax1 = ax1_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    ax2 = ax2_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    ax_ecg[sub] = mean_df2_phy(ax1, ax2)
    ax_hrv[sub] = mean_df2_phy(rain_hrv(ecg_ax1, 2000, 250), rain_hrv(ecg_ax2, 2000, 250))

## Save the anxiety result to .npy file
np.save(os.path.join(results_dir, 'ax_ecg.npy'), ax_ecg) 
np.save(os.path.join(results_dir, 'ax_hrv.npy'), ax_hrv) 

Extract features to conduct ISC

In [ ]:
ax_ecg = np.load(os.path.join(results_dir, 'ax_ecg.npy'), allow_pickle=True).item()
ax_hrv = np.load(os.path.join(results_dir, 'ax_hrv.npy'), allow_pickle=True).item()
ax_hr, ax_edr, ax_hrvm, ax_hrvsd = np.zeros((5000, 38)), np.zeros((5000, 38)), np.zeros((38)), np.zeros((38))
for sub in sub_ax:
    ## For the index 1 for 'ECG_Rate', 2 for 'EDR'
    hr = ax_ecg[sub].iloc[:,1].to_numpy()
    edr = ax_ecg[sub].iloc[:,2].to_numpy()
    mean = ax_hrv[sub].iloc[:,0]
    sd = ax_hrv[sub].iloc[:,1]
    hr = np.ndarray.flatten(hr)
    edr = np.ndarray.flatten(edr)
    idx = sub_ax.index(sub)
    ax_hr[:,idx] = hr
    ax_edr[:,idx] = edr
    ax_hrvm[idx] = mean
    ax_hrvsd[idx] = sd

 Calculate the ISC on anxiety ECG features

In [ ]:
# Calculate ISC on angry heart rate
ax_hr_similarity, ax_hr_stats = isc(ax_hr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
ax_hr_similarity = ax_hr_similarity.squareform()
ax_edr_similarity, ax_edr_stats = isc(ax_edr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
ax_edr_similarity = ax_edr_similarity.squareform()

# Plot
mask =np.zeros_like(ax_hr_similarity)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(ax_hr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Anxiety HR ISC", fontsize=25, fontweight='bold')
sns.heatmap(ax_edr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Anxiety EDR ISC", fontsize=25, fontweight='bold')

# Save the result
np.save(os.path.join(results_dir, 'ax_hr_isc.npy'), ax_hr_similarity) 
np.save(os.path.join(results_dir, 'ax_edr_isc.npy'), ax_edr_similarity) 

Calculate ISC on Anxiety HRV features

In [ ]:
# Set the paprameters and import data
n_subs = 38
hrvm_rank, hrvsd_rank = rankdata(ax_hrvm), rankdata(ax_hrvsd) # explicity convert the raw scores to ranks
hrvm_simi, hrvsd_simi = np.zeros((n_subs, n_subs)), np.zeros((n_subs, n_subs))

# Calculate the ann isc
for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvm_rank[i], hrvm_rank[j]])/n_subs
            hrvm_simi[i,j] = sim_ij
            hrvm_simi[j,i] = sim_ij
        elif i==j:
            hrvm_simi[i,j] = 1

for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvsd_rank[i], hrvsd_rank[j]])/n_subs
            hrvsd_simi[i,j] = sim_ij
            hrvsd_simi[j,i] = sim_ij
        elif i==j:
            hrvsd_simi[i,j] = 1

# Plot the isc matrix
mask =np.zeros_like(hrvm_simi)
mask[np.triu_indices_from(mask)] = True
# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
# sns.heatmap(hrvm_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax1.set_title("Anxiety HRV mean ISC", fontsize=25, fontweight='bold')
# sns.heatmap(hrvsd_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax2.set_title("Anxiety HRV SD ISC", fontsize=25, fontweight='bold')

plt.figure(figsize=(20,10))
sns.heatmap(hrvm_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
plt.title('Anxiety HRV Intersubject Similarity', fontsize=25, fontweight='bold')
# Save the result
# np.save(os.path.join(results_dir, 'ax_hrvm_isc.npy'), hrvm_simi) 
# np.save(os.path.join(results_dir, 'ax_hrvsd_isc.npy'), hrvsd_simi) 

## Section 4: ISC for *Fear* ECG

* fe1: 110 - 130
* fe2: 30 - 50
* fe3: 45 - 65

## Preprocess the raw ECG and aggregate results
* Features: Cleaned signal, Heart Rate, ECG derived Respiration(EDR), HVR_MeanNN, HRV_SDNN

In [ ]:
# Construct a meta dict for angry results
fe_ecg, fe_hrv = {},{}
# Loop for all subjects
for sub in sub_fe:
    ecg_fe1, ecg_fe2, ecg_fe3 = fe1_meta[sub][110*2000:130*2000], fe2_meta[sub][30*2000:50*2000], fe3_meta[sub][45*2000:65*2000]
    fe1_results, fe2_results, fe3_results = rain_ecg(ecg_fe1, 2000, 250), rain_ecg(ecg_fe2, 2000, 250), rain_ecg(ecg_fe3, 2000, 250)
    fe1 = fe1_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    fe2 = fe2_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    fe3 = fe3_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    fe_ecg[sub] = mean_df3_phy(fe1, fe2, fe3)
    fe_hrv[sub] = mean_df3_phy(rain_hrv(ecg_fe1, 2000, 250), rain_hrv(ecg_fe2, 2000, 250), rain_hrv(ecg_fe3, 2000, 250))

## Save the fear result to .npy file
np.save(os.path.join(results_dir, 'fe_ecg.npy'), fe_ecg) 
np.save(os.path.join(results_dir, 'fe_hrv.npy'), fe_hrv) 

Extract features to conduct ISC

In [ ]:
fe_ecg = np.load(os.path.join(results_dir, 'fe_ecg.npy'), allow_pickle=True).item()
fe_hrv = np.load(os.path.join(results_dir, 'fe_hrv.npy'), allow_pickle=True).item()
fe_hr, fe_edr, fe_hrvm, fe_hrvsd = np.zeros((5000, 37)), np.zeros((5000, 37)), np.zeros((37)), np.zeros((37))
for sub in sub_fe:
    hr = fe_ecg[sub].iloc[:,1].to_numpy()
    edr = fe_ecg[sub].iloc[:,2].to_numpy()
    mean = fe_hrv[sub].iloc[:,0]
    sd = fe_hrv[sub].iloc[:,1]
    hr = np.ndarray.flatten(hr)
    edr = np.ndarray.flatten(edr)
    idx = sub_fe.index(sub)
    fe_hr[:,idx] = hr
    fe_edr[:,idx] = edr
    fe_hrvm[idx] = mean
    fe_hrvsd[idx] = sd

 Calculate the ISC on fear ECG features

In [ ]:
# Calculate ISC on angry heart rate
fe_hr_similarity, fe_hr_stats = isc(fe_hr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
fe_hr_similarity = fe_hr_similarity.squareform()
fe_edr_similarity, fe_edr_stats = isc(fe_edr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
fe_edr_similarity = fe_edr_similarity.squareform()

# Plot
mask =np.zeros_like(fe_hr_similarity)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(fe_hr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Fear HR ISC", fontsize=25, fontweight='bold')
sns.heatmap(fe_edr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Fear EDR ISC", fontsize=25, fontweight='bold')

# Save the result
np.save(os.path.join(results_dir, 'fe_hr_isc.npy'), fe_hr_similarity) 
np.save(os.path.join(results_dir, 'fe_edr_isc.npy'), fe_hr_similarity) 

 Conduct ISC on fear HRV features

In [ ]:
# Set the paprameters and import data
n_subs = 37
hrvm_rank, hrvsd_rank = rankdata(fe_hrvm), rankdata(fe_hrvsd) # explicity convert the raw scores to ranks
hrvm_simi, hrvsd_simi = np.zeros((n_subs, n_subs)), np.zeros((n_subs, n_subs))

# Calculate the ann isc
for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvm_rank[i], hrvm_rank[j]])/n_subs
            hrvm_simi[i,j] = sim_ij
            hrvm_simi[j,i] = sim_ij
        elif i==j:
            hrvm_simi[i,j] = 1

for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvsd_rank[i], hrvsd_rank[j]])/n_subs
            hrvsd_simi[i,j] = sim_ij
            hrvsd_simi[j,i] = sim_ij
        elif i==j:
            hrvsd_simi[i,j] = 1

# Plot the isc matrix
mask =np.zeros_like(hrvm_simi)
mask[np.triu_indices_from(mask)] = True
# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
# sns.heatmap(hrvm_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax1.set_title("Fear HRV mean ISC", fontsize=25, fontweight='bold')
# sns.heatmap(hrvsd_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax2.set_title("Fear HRV SD ISC", fontsize=25, fontweight='bold')

plt.figure(figsize=(20,10))
sns.heatmap(hrvm_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
plt.title('Fear HRV Intersubject Similarity', fontsize=25, fontweight='bold')
# Save the result
# np.save(os.path.join(results_dir, 'fe_hrvm_isc.npy'), hrvm_simi) 
# np.save(os.path.join(results_dir, 'fe_hrvsd_isc.npy'), hrvsd_simi) 

## Section 5: ISC for *Helpless* ECG

* hl2: 115 - 135, The astronaut has run out of oxygen and she managed to broke her arm, 
* hl3: 50 - 70, The woman ask for the man for medicine 

Preprocess the raw ECG and aggregate results
* Features: Cleaned signal, Heart Rate, ECG derived Respiration(EDR), HVR_MeanNN, HRV_SDNN

In [ ]:
# Construct a meta dict for anxiety results
hl_ecg, hl_hrv = {},{}
# Loop for all subjects
for sub in sub_hl:
    ecg_hl2, ecg_hl3 = hl2_meta[sub][115*2000:135*2000], hl3_meta[sub][50*2000:70*2000]
    hl2_results, hl3_results = rain_ecg(ecg_hl2, 2000, 250), rain_ecg(ecg_hl3, 2000, 250)
    hl2 = hl2_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    hl3 = hl3_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    hl_ecg[sub] = mean_df2_phy(hl2, hl3)
    hl_hrv[sub] = mean_df2_phy(rain_hrv(ecg_hl2, 2000, 250), rain_hrv(ecg_hl3, 2000, 250))

## Save the anxiety result to .npy file
np.save(os.path.join(results_dir, 'hl_ecg.npy'), hl_ecg) 
np.save(os.path.join(results_dir, 'hl_hrv.npy'), hl_hrv) 

Extract features to conduct ISC

In [ ]:
hl_ecg = np.load(os.path.join(results_dir, 'hl_ecg.npy'), allow_pickle=True).item()
hl_hrv = np.load(os.path.join(results_dir, 'hl_hrv.npy'), allow_pickle=True).item()
hl_hr, hl_edr, hl_hrvm, hl_hrvsd = np.zeros((5000, 39)), np.zeros((5000, 39)), np.zeros((39)), np.zeros((39))
for sub in sub_hl:
    ## For the index 1 for 'ECG_Rate', 2 for 'EDR'
    hr = hl_ecg[sub].iloc[:,1].to_numpy()
    edr = hl_ecg[sub].iloc[:,2].to_numpy()
    mean = hl_hrv[sub].iloc[:,0]
    sd = hl_hrv[sub].iloc[:,1]
    hr = np.ndarray.flatten(hr)
    edr = np.ndarray.flatten(edr)
    idx = sub_hl.index(sub)
    hl_hr[:,idx] = hr
    hl_edr[:,idx] = edr
    hl_hrvm[idx] = mean
    hl_hrvsd[idx] = sd

Conduct ISC on helpless ECG features

In [ ]:
# Calculate ISC on angry heart rate
hl_hr_similarity, hl_hr_stats = isc(hl_hr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
hl_hr_similarity = hl_hr_similarity.squareform()
hl_edr_similarity, hl_edr_stats = isc(hl_edr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
hl_edr_similarity = hl_edr_similarity.squareform()

# Plot
mask =np.zeros_like(hl_hr_similarity)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(hl_hr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Helpless HR Intersubject Similarity", fontsize=20, fontweight='bold')
sns.heatmap(hl_edr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Helpless EDR Intersubject Similarity", fontsize=20, fontweight='bold')

# Save the result
# np.save(os.path.join(results_dir, 'hl_hr_isc.npy'), hl_hr_similarity) 
# np.save(os.path.join(results_dir, 'hl_edr_isc.npy'), hl_edr_similarity) 


Conduct ISC on helpless HRV features

In [ ]:
# Set the paprameters and import data
n_subs = 39
hrvm_rank, hrvsd_rank = rankdata(hl_hrvm), rankdata(hl_hrvsd) # explicity convert the raw scores to ranks
hrvm_simi, hrvsd_simi = np.zeros((n_subs, n_subs)), np.zeros((n_subs, n_subs))

# Calculate the ann isc
for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvm_rank[i], hrvm_rank[j]])/n_subs
            hrvm_simi[i,j] = sim_ij
            hrvm_simi[j,i] = sim_ij
        elif i==j:
            hrvm_simi[i,j] = 1

for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvsd_rank[i], hrvsd_rank[j]])/n_subs
            hrvsd_simi[i,j] = sim_ij
            hrvsd_simi[j,i] = sim_ij
        elif i==j:
            hrvsd_simi[i,j] = 1

# Plot the isc matrix
mask =np.zeros_like(hrvm_simi)
mask[np.triu_indices_from(mask)] = True
# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
# sns.heatmap(hrvm_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax1.set_title("Fear HRV mean ISC", fontsize=25, fontweight='bold')
# sns.heatmap(hrvsd_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax2.set_title("Fear HRV SD ISC", fontsize=25, fontweight='bold')

plt.figure(figsize=(20,10))
sns.heatmap(hrvm_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
plt.title('Helpless HRV Intersubject Similarity', fontsize=25, fontweight='bold')
# Save the result
# np.save(os.path.join(results_dir, 'hl_hrvm_isc.npy'), hrvm_simi) 
# np.save(os.path.join(results_dir, 'hl_hrvsd_isc.npy'), hrvsd_simi) 

## Section 6: ISC for *happy* ECG

* ha1: 180:200
* ha3: 5-25

Process raw ECG and aggregate results
* Features: Cleaned signal, Heart Rate, ECG derived Respiration(EDR), HVR_MeanNN, HRV_SDNN

In [ ]:
# Construct a meta dict for happy results
ha_ecg, ha_hrv = {},{}
# Loop for all subjects
for sub in sub_ha:
    ecg_ha1, ecg_ha3 = ha1_meta[sub][180*2000:200*2000], ha3_meta[sub][5*2000:25*2000]
    ha1_results, ha3_results = rain_ecg(ecg_ha1, 2000, 250), rain_ecg(ecg_ha3, 2000, 250)
    ha1 = ha1_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    ha3 = ha3_results.loc[:,['ECG_Clean', 'ECG_Rate', 'ECG_EDR']]
    ha_ecg[sub] = mean_df2_phy(ha1, hl3)
    ha_hrv[sub] = mean_df2_phy(rain_hrv(ecg_ha1, 2000, 250), rain_hrv(ecg_ha3, 2000, 250))

## Save the happy result to .npy file
np.save(os.path.join(results_dir, 'ha_ecg.npy'), ha_ecg) 
np.save(os.path.join(results_dir, 'ha_hrv.npy'), ha_hrv) 

Extract features to conduct ISC

In [ ]:
ha_ecg = np.load(os.path.join(results_dir, 'ha_ecg.npy'), allow_pickle=True).item()
ha_hrv = np.load(os.path.join(results_dir, 'ha_hrv.npy'), allow_pickle=True).item()
ha_hr, ha_edr, ha_hrvm, ha_hrvsd = np.zeros((5000, 41)), np.zeros((5000, 41)), np.zeros((41)), np.zeros((41))
for sub in sub_ha:
    ## For the index 1 for 'ECG_Rate', 2 for 'EDR'
    hr = ha_ecg[sub].iloc[:,1].to_numpy()
    edr = ha_ecg[sub].iloc[:,2].to_numpy()
    mean = ha_hrv[sub].iloc[:,0]
    sd = ha_hrv[sub].iloc[:,1]
    hr = np.ndarray.flatten(hr)
    edr = np.ndarray.flatten(edr)
    idx = sub_ha.index(sub)
    ha_hr[:,idx] = hr
    ha_edr[:,idx] = edr
    ha_hrvm[idx] = mean
    ha_hrvsd[idx] = sd

Conduct ISC on happy ECG features

In [ ]:
# Calculate ISC on angry heart rate
ha_hr_similarity, ha_hr_stats = isc(ha_hr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
ha_hr_similarity = ha_hr_similarity.squareform()
ha_edr_similarity, ha_edr_stats = isc(ha_edr, n_bootstraps=5000, metric='median', method='bootstrap', n_jobs=-1)
ha_edr_similarity = ha_edr_similarity.squareform()

# Plot
mask =np.zeros_like(ha_hr_similarity)
mask[np.triu_indices_from(mask)] = True
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
sns.heatmap(ha_hr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax1.set_title("Happy HR ISC", fontsize=25, fontweight='bold')
sns.heatmap(ha_edr_similarity, square=True, cmap='RdBu_r', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
ax2.set_title("Happy EDR ISC", fontsize=25, fontweight='bold')

# Save the result
np.save(os.path.join(results_dir, 'ha_hr_isc.npy'), ha_hr_similarity) 
np.save(os.path.join(results_dir, 'ha_edr_isc.npy'), ha_edr_similarity) 


Conduct ISC on happy HRV features

In [ ]:
# Set the paprameters and import data
n_subs = 41
hrvm_rank, hrvsd_rank = rankdata(ha_hrvm), rankdata(ha_hrvsd) # explicity convert the raw scores to ranks
hrvm_simi, hrvsd_simi = np.zeros((n_subs, n_subs)), np.zeros((n_subs, n_subs))

# Calculate the ann isc
for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvm_rank[i], hrvm_rank[j]])/n_subs
            hrvm_simi[i,j] = sim_ij
            hrvm_simi[j,i] = sim_ij
        elif i==j:
            hrvm_simi[i,j] = 1

for i in range(n_subs):
    for j in range(n_subs):
        if i < j:
            sim_ij = np.mean([hrvsd_rank[i], hrvsd_rank[j]])/n_subs
            hrvsd_simi[i,j] = sim_ij
            hrvsd_simi[j,i] = sim_ij
        elif i==j:
            hrvsd_simi[i,j] = 1

# Plot the isc matrix
mask =np.zeros_like(hrvm_simi)
mask[np.triu_indices_from(mask)] = True
# fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,8))
# sns.heatmap(hrvm_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax1.set_title("Fear HRV mean ISC", fontsize=25, fontweight='bold')
# sns.heatmap(hrvsd_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, ax=ax2, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
# ax2.set_title("Fear HRV SD ISC", fontsize=25, fontweight='bold')

plt.figure(figsize=(20,10))
sns.heatmap(hrvm_simi, square=True, cmap='Reds', mask=mask, linewidths=0.1, xticklabels=False, yticklabels=False, cbar_kws={'label': 'similarity', "shrink": 0.8})
plt.title('Happy HRV Intersubject Similarity', fontsize=25, fontweight='bold')
# Save the result
# np.save(os.path.join(results_dir, 'ha_hrvm_isc.npy'), hrvm_simi) 
# np.save(os.path.join(results_dir, 'ha_hrvsd_isc.npy'), hrvsd_simi) 